In [1]:
#環境の定義

__credits__ = ["Kallinteris-Andreas"]

import os
import warnings
import time
import matplotlib.pyplot as plt
import mujoco
import mujoco.viewer
import glfw
# import keyboard
import math
import numpy as np
import pytest
import random
from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env


class MyCartPoleEnv(MujocoEnv, utils.EzPickle):
    """
    Gymansium.MujocoEnv`環境APIを使った，倒立振子の強化学習環境\n
    倒れないで立ち続けるタスク．\n
    mujoco envを継承しています．
    """

    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="my_xmls/mycartpole.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            camera_name="camera_name",
            **kwargs,
        )

        self.metadata = { #またメタデータがある。インスタンス変数？
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = 6 #self.data.qpos.size + self.data.qvel.size #たとえば，観測空間に位置と速度を入れたいのであれば，その数をここに入れる。次元数なので数え方は０始まりではない。
        # self.wall_hit = 0
        #マイクロマウスのパラメータ用変数の用意
        self.reward_graph = []
        self.epi_reward_graph = []
        self.mouse_xpos_graph = []
        self.mouse_ypos_graph = []
        self.lastxpos_graph = []
        self.lastypos_graph = []
        self.mouse_angle_rad = math.pi/2
        self.mouse_xpos = 0
        self.mouse_ypos = 0
        self.mouse_vel = 0
        self.now_time = 0
        self.past_time = 0
        self.delta_t = 0.01
        self.wheel_r = 0
        self.gear = 9.0e-3
        self.wheel_r = 0.0135
        self.tread = 0.072
        self.right_rotation_sum = 0
        self.left_rotation_sum = 0
        self.stepcount = 0
        self.angsum_obs = 0
        self.epi_reward = 0
        self.obsLS_graph = []
        self.obsLF_graph = []
        self.obsRS_graph = []
        self.obsRF_graph = []
        self.range_reward_x_graph = []
        self.range_reward_y_graph = []
        self.xpos_graph = []
        self.ypos_graph = []
        self.firstxyzahyou = self.init_qpos[0:3]
        self.firstquat = self.init_qpos[3:7]
        self.first_ang = (0, 0, 90)
        self.first_ang_graph = []
        


        self.observation_space = Box( #観測空間を定義．gymnasium.sapcesのBoxを使用することで簡単に定義できる．Boxは連続値を扱う．
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64 #float64型の，-無限から無限までの値を取る，11次元の観測空間を用意
        )

    # 
    def get_odom(self, model, data): #今のタイヤの回転角度と速度（角度と角速度）を取得．
        """
            現在のタイヤの回転角度と速度（と角速度）を取得する．\\
            data.actuator().length,velocityで取得できる値はactuatorのgear倍されているので，それで割ることによって打ち消す，
            Returns:
                odm_right = 右タイヤの回転角度[rad]（基準からの総量） \\
                odm_left = 左タイヤの回転角度[rad]（基準からの総量） \\
                (vel_left) = 左タイヤの回転角速度 (単位時間あたり) \\
            """
        odm_right = data.actuator('right').length[0]/self.gear # 得た値をgearで割ったもの＝タイヤの角度(rad)
        odm_left = data.actuator('left').length[0]/self.gear # ＝タイヤの角度
        vel_left = data.actuator('left').velocity[0]/self.gear # ＝タイヤの回転角速度
        return odm_right, odm_left
    
    def get_pulse_count(self, pre_odm_right, pre_odm_left, odm_right, odm_left): #回転量をパルスにする エンコーダの役割をする関数
        """
            回転量の差から，出たパルスの数を求める． (エンコーダの役割をする関数)\\
            
            Returns:
                right_pulse_num = 右タイヤのパルス数 \\
                left_pulse_num = 左タイヤのパルス数
            """
        
        pre_right_rotation_sum = self.right_rotation_sum #１つまえの回転量取得
        pre_left_rotation_sum = self.left_rotation_sum
        self.right_rotation_sum += (odm_right - pre_odm_right)/(2*math.pi) *4096 #1周あたり4096段階で，細かい回転量を計測（分解能ほぼ無限のエンコーダ）（小数で、0から1で回転を表していたものを0から4096で表すようにした）
        self.left_rotation_sum += (odm_left - pre_odm_left)/(2*math.pi) *4096 #1周あたり4096段階で，細かい回転量を計測（分解能ほぼ無限のエンコーダ）
        right_pulse_num = int(self.right_rotation_sum) - int(pre_right_rotation_sum) #現在の回転量を足した後の回転合計値ー足す前の回転合計値　これのintが１以上なら、１パルス出たことになる。パルスが出なくても回転量の合計は溜まっていく
        left_pulse_num = int(self.left_rotation_sum) - int(pre_left_rotation_sum)
        # print("回転量をパルス段階に変換　この値の差が，実際に出たパルスとなる：", right_pulse_num)
        return right_pulse_num, left_pulse_num #出たパルスの数
    
    def get_odom_at_pulse(self, right_pulse_num, left_pulse_num): #パルス数を元に、パルス数を元に、ロボットの速度と回転角速度を求める（オドメトリによる推定を行う）．
        """
            パルス数を元に、ロボットの速度と回転角速度を求める．（オドメトリによる推定を行う）\\
            Returns:
                mouse_vel = ロボットの推定速度 \\
                left_pulse_num = ロボットの推定回転速度
            """
        right_wheel_move = 2*math.pi * self.wheel_r * (right_pulse_num/4096) #２πｒ×回転数で移動距離を求める．/delta_t
        left_wheel_move = 2*math.pi * self.wheel_r * (left_pulse_num/4096) #２πｒ×回転数で移動距離を求める．
        right_wheel_vel = right_wheel_move/self.delta_t #移動距離から，速度を求める
        left_wheel_vel = left_wheel_move/self.delta_t #移動距離から，速度を求める
        mouse_vel = (right_wheel_vel + left_wheel_vel)/2 #左右平均が並進速度
        mouse_radvel = (right_wheel_vel - left_wheel_vel)/self.tread #これがマウスの旋回角速度（rad/s）
        return mouse_vel, mouse_radvel #これで，エンコーダによる各タイヤの推定速度が求められた
    

    def step(self, action):
        """
          シミュレーションを1step進める。 \\
          Returns: \\
            observation, 観測空間。その行動後のエージェントが見えている状態を返す \\
            reward, 報酬の合計値 \\
            terminated = False, タスクの早期達成または強制終了条件にひっかかったわけではないが、意味のない行動をし続ける場合に陥ったときなどにエピソードを終了するフラグ。本環境ではそういう場面はないため常にFalseとなる。 \\
            truncated = タスクの早期達成または強制終了が起きたときにそれを示すもの。壁にぶつかったら強制終了が起きる。ぶつかったとき以外はFalse。 \\
            info = （現状未確認。何かの情報が入っている？）\\
        """
        #変数定義
        terminated = False
        wall_hit = 0
        course_out = 0
        goal = 0
        died_reward = 0
        # velocity_before = self.data.sensor("Veloci").data[0] # シミュレーションを進める前の速度センサの値を取得
        ####オドメトリ####
        self.past_time = self.data.time #シミュレーション進める前の時間を取得
        pre_odm_right, pre_odm_left = self.get_odom(self.model, self.data) #今のタイヤの回転角度と速度（角度と角速度）を取得．
        
        self.do_simulation(action, self.frame_skip)####################シミュレーション実行####################################################################################
        self.now_time = self.data.time #1step進んだ後のシミュレーションの時間を取得
        self.delta_t = self.now_time - self.past_time #シミュレーション1ステップにかかった時間を取得
        self.past_time = self.now_time #シミュレーション前の時間の変数を更新
        odm_right, odm_left = self.get_odom(self.model, self.data)# エンコーダをもう一度読み取る　（行動後の値を取得）
        
        right_pulse_num, left_pulse_num = self.get_pulse_count(pre_odm_right, pre_odm_left, odm_right, odm_left)# シミュレーション前後のエンコーダ値から，この一瞬に出たパルス数を算出し，
        mouse_vel, mouse_rad_vel = self.get_odom_at_pulse(right_pulse_num, left_pulse_num) # パルス数から，移動速度と角速度の推定値を求める。　これで，エンコーダによるマウスの速度・角速度が推定できた
        # 速度に時間を掛けて，移動距離を計算 まず向きを計算し，その後移動距離を求める
        self.mouse_angle_rad += mouse_rad_vel * self.delta_t #マウスの角速度はパルスの左右差で求められる。現在の角度は角速度に時間をかけた角度変化量を合計していくと求まる。
        self.mouse_xpos += mouse_vel * math.cos(self.mouse_angle_rad) * self.delta_t #向いている角度と、進んでいる速度の情報から、ロボットの現在のx座標を更新する。
        self.mouse_ypos += mouse_vel * math.sin(self.mouse_angle_rad) * self.delta_t #y座標版
        self.mouse_xpos_graph.append(self.mouse_xpos)
        self.mouse_ypos_graph.append(self.mouse_ypos)
        ####オドメトリ終了####

        #velocity_after = self.data.sensor("Veloci").data[0] # 今の速度センサの値を取得
        #mouse_vel = velocity_after - velocity_before #速度センサの値による、ロボットの速度を求める,,,速度ー速度で、速度変化を求めているけどつかわなそう。なんだこれ
        # if mouse_vel < 0:
        #     3*mouse_vel
        self.mouse_vel = mouse_vel #ロボットの推定速度値をインスタンス変数に保存しておく処理

        hit = self.data.sensor("HB1").data[0] #フォースセンサの値取得
        truncated = False
        # print("hit wall", hit_wall_f, hit_wall_b)
        if hit > 0: #センサがぶつかったことを検出したら倒れたということなので、
            died_reward = 100
            truncated = True

        goal_pos = self.data.body("goal").xpos[0:2] #xy座標のみ
        robot_pos = self.data.body("torso").xpos[0:2]
        goal_range_reward = np.linalg.norm(robot_pos - goal_pos) #ゴールとの直線距離　ユークリッド距離を計算
        survive_reward = 0.3

        observation = self._get_obs(self.angsum_obs)
        reward = survive_reward - 0* goal_range_reward - died_reward 
        # reward = wall_hit + course_out + range_reward_x-0.5 +range_reward_y-0.5 + ang_reward[2]-0.5 - 1* np.sum(np.square(action)) # 目標との誤差に関する報酬は-0.5をつけて遠すぎると罰になるようオフセットを加えた。制御入力がでかいほど罰が大きくなることによって無駄な動きを抑制。 self.wall_hitにしないほうが（インスタンス変数にしないほうが）見通しが良い気がしたのでそうした。
        # print("wall_hit:",wall_hit , "course_out:", course_out, "range_reward:", range_reward, "ang_reward:", ang_reward)
        self.reward_graph.append(reward)
        self.epi_reward += reward
        info = {}
        
        
            
        if self.render_mode == "human":
            self.render()
        # print("reward", reward)
        reward = float(reward)
        return observation, reward, terminated, truncated, info

    
    def _get_obs(self, angsum_obs): #状態空間を取得 学習のループで呼ばれる用
        angsum_obs = np.array([angsum_obs])
        # position = self.data.qpos[0:7].flat.copy() #神の視点の位置情報取得


        # velocity = self.data.sensor("Veloci").data[0] # 今の速度を取る
        gyro = np.array([self.data.sensor('gyro').data])
        gyro_0 = np.array([gyro[0][0]])
        gyro_1 = np.array([gyro[0][1]])
        gyro_2 = np.array([gyro[0][2]])
        accelerometer = np.array([self.data.sensor('Accel').data])
        accelerometer_0 = np.array([accelerometer[0][0]])
        accelerometer_1 = np.array([accelerometer[0][1]])
        accelerometer_2 = np.array([accelerometer[0][2]])
        mouse_angle_rad = np.array([self.mouse_angle_rad])
        mouse_xpos = np.array([self.mouse_xpos])#オドメトリによる推定座標
        mouse_ypos = np.array([self.mouse_ypos])#オドメトリによる推定座標
        self.xpos_graph.append(mouse_xpos)
        self.ypos_graph.append(mouse_ypos)
        goal_pos = self.data.body("goal").xpos[0:2] #xy座標のみ

        # ゴールの位置と、自分のオドメトリの推定値と、各センサの値が観測できる
        # return np.concatenate((goal_pos, mouse_xpos, mouse_ypos, mouse_angle_rad, accelerometer_0, accelerometer_1, accelerometer_2, gyro_0, gyro_1, gyro_2))
        return np.concatenate((accelerometer_0, accelerometer_1, accelerometer_2, gyro_0, gyro_1, gyro_2))
        
    def reset_model(self):
        qpos = self.init_qpos.copy() #初期位置を取得
        qvel = self.init_qvel.copy() #初期速度を取得
        goal_xpos = np.random.uniform(0.2, 0.9)
        goal_ypos = np.random.uniform(0.2,0.9)
                
        qpos[-2] = goal_xpos
        qpos[-1] = goal_ypos
        self.stepcount = 0
        

        # leftwall_on = np.random.uniform(0, 1)
        # rightwall_on = np.random.uniform(0, 1)
        # forwardwall_on = np.random.uniform(0, 1)
        # backwall_on = np.random.uniform(0, 1)
        # if leftwall_on >= 0:
        #     qpos[-1] = -0.4
        # else:
        #     qpos[-1] = 0
        # if rightwall_on >= 0:
        #     qpos[-2] = 0.4
        # else:
        #     qpos[-2] = 0
        # if forwardwall_on >= 0:
        #     qpos[-3] = 0.4
        # else:
        #     qpos[-3] = 0
        # if backwall_on >= 0:
        #     qpos[-4] = -0.4
        # else:
        #     qpos[-4] = 0

        # x_values = [0.27, 1.07, 1.87]
        # mouse_xposition_shuffle = random.choice(x_values)
        # x_values = 0.27

        # mouse_xposition_shuffle = self.course_list[np.random]
        # mouse_yposition_shuffle = self.course_list[np.random]

        #両方3のときはやり直しする処理
        
        # qpos[0] = mouse_xposition_shuffle
        # qpos[0] = x_values
        # noise = np.random.normal(0, 0.3)  # 平均0、標準偏差0.1のノイズ
        # qpos[3] += np.random.normal(0, 0.03)#0.27 1.07
        # qpos[1] = mouse_yposition_shuffle
        self.set_state(qpos, qvel) #qposとqvelには，すべての位置と速度の値がならんでいるためそれをセットする#################
        self.mouse_angle_rad = 0
        self.mouse_xpos = 0
        self.mouse_ypos = 0
        self.angsum_obs = 0

        self.epi_reward_graph.append(self.epi_reward) #収益のグラフ表示用
        self.epi_reward = 0 #収益を数え終わったから０にリセットして次のエピソードの報酬を数えていくため

        observation = self._get_obs(self.angsum_obs)
        #必要に応じてオドメトリ値もリセットする必要がある

        return observation

    def _get_reset_info(self):
        return {"works": True}
    
    # from typing import Any, Dict, Optional, Tuple, Union
    # def reset(
    #     self,
    #     *,
    #     seed: Optional[int] = None,
    #     options: Optional[dict] = None,
    # ):
    #     super().reset(seed=seed) #サブクラスにresetを書くとそちらだけが実行されるように隠蔽されてしまうので，スーパークラスのresetを実行して，前に定義した内容も実行する

    #     self._reset_simulation()
    #     print("リセットメソッドのオーバーライドができているか確認するためのprint文です．")

    #     ob = self.reset_model()
    #     info = self._get_reset_info()

    #     if self.render_mode == "human":
    #         self.render()
    #     return ob, info

#＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃オプション。特に使わない＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃
CHECK_ENV_IGNORE_WARNINGS = [
    f"\x1b[33mWARN: {message}\x1b[0m"
    for message in [
        "A Box observation space minimum value is -infinity. This is probably too low.",
        "A Box observation space maximum value is infinity. This is probably too high.",
        "For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.",
    ]
]


@pytest.mark.parametrize("frame_skip", [1, 2, 3, 4, 5])
def test_frame_skip(frame_skip):
    """verify that custom envs work with different `frame_skip` values"""
    env = MyCartPoleEnv(frame_skip=frame_skip) #フレームスキップ設定の上書き？

    # Test if env adheres to Gym API
    with warnings.catch_warnings(record=True) as w:
        check_env(env.unwrapped, skip_render_check=True)
        env.close()
    for warning in w:
        if warning.message.args[0] not in CHECK_ENV_IGNORE_WARNINGS:
            raise Error(f"Unexpected warning: {warning.message}")


def test_xml_file():
    """Verify that the loading of a custom XML file works \\
      訳：XMLファイルの読み込みがうまくいくことを確認する"""
    relative_path = "./tests/envs/mujoco/assets/walker2d_v5_uneven_feet.xml"
    env = MyCartPoleEnv(xml_file=relative_path).unwrapped
    assert isinstance(env, MujocoEnv)
    assert env.data.qpos.size == 9

    full_path = os.getcwd() + "/tests/envs/mujoco/assets/walker2d_v5_uneven_feet.xml"
    env = MyCartPoleEnv(xml_file=full_path).unwrapped
    assert isinstance(env, MujocoEnv)
    assert env.data.qpos.size == 9

    # note can not test user home path (with '~') because github CI does not have a home folder　訳：ユーザーホームパス（'〜'を含む）をテストできないことに注意してください。github CIにはホームフォルダがありません


def test_reset_info():
    """Verify that the environment returns info at `reset()`"""
    env = MyCartPoleEnv()

    _, info = env.reset()
    assert info["works"] is True
#＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃オプションここまで＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃

#＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃用意した環境クラスを使用し、学習済みモデルをロード＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃

import gymnasium as gym
gym.envs.registration.register(id='MyCartPoleEnv-v0',max_episode_steps=1000,entry_point=MyCartPoleEnv) #環境登録 idは環境の名前付け。 max_episode_steps=600, は1エピソードの最大ステップ数 entry_point=MouseRightEnv は環境クラスの指定

from stable_baselines3 import SAC, PPO
from tqdm import trange
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from gymnasium.wrappers import RecordVideo
import datetime

from stable_baselines3.common.callbacks import BaseCallback

start_time = datetime.datetime.now().replace(microsecond=0) #学習にかかった時間を計測するために使用　小数の時間は切り捨て(microsecond=0)

env = gym.make("MyCartPoleEnv-v0", render_mode="human") #環境クラスの作成。レンダーモードを選択 







#Create model
# model = mujoco.MjModel.from_xml_path('../mouse_in_maze.xml')
# data = mujoco.MjData(model)
model = env.model
data = env.data

odom_right_graph = []
mouse_correct_vel_graph = []

gear = 9.0e-3
wheel_r = 0.0135
tread = 0.072
mx = 0.0
my = 0.0
psi = 0.0
past_odom_right = 0.0
past_odom_left = 0.0
# kp = 100
# st_Kp = 1.55 #ダンピング3.6e-6のときの最適なゲイン
st_Kp = 8.4
rad_Kp = 0.15
rad_vel_Kp = 2

rad_Kd = 0.00001

ki6 = 0.02
kd = 0.09
I_err_sum = 0
d_grad = 0
prev_err_rad = 0
target_vel = 0
target_rad = math.pi/2
target_rad_vel = 0
mjstep_count = 0

right_pulse_num = 0
right_pulse_num_graph = []
left_pulse_num = 0
left_pulse_num_graph = []

right_rotation_sum = 0
left_rotation_sum = 0

mouse_vel = 0
mouse_rad_vel = 0
mouse_xpos = 0
mouse_xpos_graph = []
mouse_ypos = 0
mouse_ypos_graph = []
target_vel_graph = []
err_vel_graph = []
mouse_vel_graph = []

viz_now = 0.0
viz_past = 0.0
now_time = 0
past_time = 0
turn_flag = 0
x_vel = 0
y_vel = 0
z_vel = 0
err = 0
delta_t = 0.001 #############################################制御周期を変える場合はここも変える！！！！！！
mouse_now_rad_xy = math.pi/2 #y軸方向を向いている
I_err_rad_vel_sum = 0
mouse_angle_rad = (math.pi/2)
right_mot = 0
left_mot = 0 #最初は0

#デバッグ用グラフのためのリスト
LRF_lf_graph = []
LRF_ls_graph = []
LRF_rs_graph = []
LRF_rf_graph = []
lf_sensor_graph = []
rf_sensor_graph = []
ls_sensor_graph = []
rs_sensor_graph = []
t_graph = []
kakusokudo_graph = []
rlkakusokudo_graph = []
kakudo_graph = []
final_kakudo_graph = []
rlfinal_kakudo_graph = []
rlkakudo_graph = []
right_mot_graph = []
left_mot_graph = []
x_vel_graph = []
velocimeter_graph = []
err_graph = []
err_rad_graph = []
now_kakudo_graph = []
now_kakudo_god_graph = []
kakudo_err_graph = []
target_kakudo_graph = []
gyro_graph = []
yzahyou = []
timevals = []
timevals5 = []
timevals6 = []
mouse_angle_rad_graph = []
mouse_correct_rad_vel_graph = []
mouse_rad_vel_graph = []
err_rad_vel_graph = []
target_rad_vel_graph = []


c:\Users\atusi\anaconda3\envs\gymenv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.model to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.model` for environment variables or `env.get_wrapper_attr('model')` that will search the reminding wrappers.
  logger.warn(
c:\Users\atusi\anaconda3\envs\gymenv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.data` for environment variables or `env.get_wrapper_attr('data')` that will search the reminding wrappers.
  logger.warn(


In [8]:
# 学習
import gymnasium as gym
from stable_baselines3 import SAC, PPO ,DQN
# from sb3_contrib import RecurrentPPO
from tqdm import trange
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo
import datetime

# env = gym.make("ALE/Breakout-v5",  render_mode="human")
# env = gym.make("CartPole-v1",  render_mode="human")
# # env = Monitor(env, "./gym-results", force=True, video_callable=lambda episode: True)　こんな感じで，
# model = SAC("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=200)

from stable_baselines3.common.callbacks import BaseCallback

start_time = datetime.datetime.now().replace(microsecond=0)

env = gym.make("MyCartPoleEnv-v0",render_mode="human")

# video_path = "./"  # 保存先のpath
# env = RecordVideo(env, video_path, video_length=500)
# env.model.opt.timestep = 0.01  # タイムステップを設定 RecordVideoする場合はそれ以降に書かないとerrorが出る

# model = PPO("MlpPolicy", env, verbose=0)

model = PPO.load('cartpoletest4',env)
# render_freq = 1000  # 1000ステップごとにレンダリング
# callback = CustomCallback(render_freq)
for i in range(10000):
    model.learn(total_timesteps=20000)#progress_bar = True, callback=callback)

#     # モデルのテスト
#     obs, _ = env.reset()  # ここを修正
    
#     for unk in trange(10000):
#         # 環境の描画
#         env.render()

#         # モデルの推論
#         # print("本物のobs!",obs)
#         action, _ = model.predict(obs)
#         # print("本物のaction", action)

#         # 1ステップ実行
#         obs, reward, terminated, truncated, info = env.step(action) #以前の4要素のタプルから5要素のタプルに変更され，`observation, reward, terminated, truncated, info`という形式になった。
#         # print(obs)


#         # エピソード完了（終了または切り捨て）のチェック terminated(終了した)は目的を達成してエピソードを終了したことを表す，truncated(切り捨てられた)は，達成できずにエピソードが終了したことを表す
#         if terminated or truncated:
#             # print(obs)
#             obs, _ = env.reset() # エピソードが終了したら、環境をリセット


            
    # #モデルの保存 (1)
    model.save('cartpoletest4')
    print("モデルを保存しました")
        # グラフを作成
    plt.plot(env.epi_reward_graph)
    plt.show()
    # model.save('hidarimawari_housyuu_godsight_onlyxy_notight')

# print(video_path)
plt.title("各ステップの報酬値", fontname="MS Gothic")
plt.grid()
plt.xlabel("ステップ数", fontname="MS Gothic")
plt.ylabel("報酬値", fontname="MS Gothic")

#前進関係のプロット
t = list(range(len(env.reward_graph)))  # 0から始まるインデックスのリストを作成 エピソードの数分報酬（収益）がリストに保存されるので、その数を数えてリストにしただけ　データ数なら自動で出るから必要なかった
plt.plot(t, env.reward_graph, linestyle='solid', label="報酬")
plt.legend()
# 環境のクローズ
env.close()

FileNotFoundError: [Errno 2] No such file or directory: 'cartpoletest4.zip'

In [3]:
import gymnasium as gym
import mujoco
from stable_baselines3 import PPO
import imageio
from tqdm import trange

def create_simulation_video(env_id, learned_model_path, timestep_interval, video_framerate, total_time, video_speed_level):
    """
    Stable baselines3で学習させたモデルのmujoco上の推論の動画を作成する関数
    Args:
        env_id (str): 環境ID
        learned_model_path (str): 学習済みモデルのパス
        timestep_interval (float): シミュレーションのタイムステップ間隔 (秒) xmlに定義したtimestepに合わせる
        video_framerate (int): ビデオのフレームレート (FPS)
        total_time (float): シミュレーションの総時間 (秒)
        video_speed_level (int): ビデオの再生速度 (倍)
        
    Returns:
        env (gym.Env): 環境

        使用例:
        env = create_simulation_video(
            env_id='MouseRightEnv-v0', #env_id='YourEnvironmentID',
            learned_model_path='test', #model_path='your_model_path.zip', 
            timestep_interval=0.01,  # シミュレーションのタイムステップ間隔 (秒) xmlに定義したtimestepに合わせる
            video_framerate=30,      # ビデオのフレームレート (FPS)
            total_time=14.0,         # シミュレーションの総時間 (秒)
            video_speed_level=3      # ビデオの再生速度 (倍) 実時間に対して何倍の速さで再生するかを決める。1倍ならそのまま、2倍なら2倍速で再生する
        )
    """
    # 環境とモデルのロード
    env = gym.make(env_id, render_mode="rgb_array")
    model = PPO.load(learned_model_path)
    
    

    # 環境の初期化
    obs, _ = env.reset()  # ここを修正
    frames = []

    # 設定
    total_steps = int(total_time / timestep_interval)  # timestepと実時間の要求から総ステップ数を計算 
    frame_capture_interval = int(1 / (timestep_interval * video_framerate))  # フレーム取得間隔を計算 要求のfpsの動画を作るには何秒ごとにスクショすればいいか計算している
    frame_capture_interval = frame_capture_interval*video_speed_level  #動画の実時間に対して何倍の速さで再生するかを決める。1倍ならそのまま、2倍なら2倍速で再生する

    # シミュレーション
    for step in trange(total_steps):
        # action, _states = model.predict(obs) #deterministicはTrueにすると、より決定論的になるそうです(?)
        action = [0.05, 0.05]  # ここを修正
        # print("action:", action)
        obs, rewards, terminated, truncated, info = env.step(action)
        if step % frame_capture_interval == 0:
            #シミュレーション内の様子をレンダーにセット
            frame = env.render() #実際にセットしたやつの見た目を描画つまりデータで表す。そしてその描画データ（スクショ）をframe変数に保存
            frames.append(frame) #ためていく
        if truncated == True or terminated == True:
            obs, _ = env.reset()

    # ビデオの保存
    with imageio.get_writer('simulation_video.mp4', fps=video_framerate) as video:
        for frame in frames:
            video.append_data(frame)

    env.close()
    return env

# 使用
env = create_simulation_video(
    env_id='MyCartPoleEnv-v0', #env_id='YourEnvironmentID',
    learned_model_path='cartpoletest', #model_path='your_model_path.zip', 
    timestep_interval=0.01,  # シミュレーションのタイムステップ間隔 (秒)
    video_framerate=10,       # ビデオのフレームレート (FPS)
    total_time=15.0,           # シミュレーションの総時間 (秒)
    video_speed_level=1       # ビデオの再生速度 (倍) 実時間に対して何倍の速さで再生するかを決める。1倍ならそのまま、2倍なら2倍速で再生する
)


100%|██████████| 1500/1500 [00:01<00:00, 765.10it/s] 


In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
else:
    print("CUDA is not available. Using CPU.")



CUDA is available. GPU will be used.


In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.3.0+cu121
True


In [1]:
import gymnasium as gym
from stable_baselines3 import PPO

env = gym.make("InvertedDoublePendulum-v4",  render_mode="human")
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000, progress_bar=True)



# モデルのテスト
state = env.reset()
for i in range(4000):
    # 環境の描画
    env.render()

    # モデルの推論
    action, _ = model.predict(state)

    # 1ステップ実行
    state, rewards, done, info = env.step(action)

    # エピソード完了
    if done:
        break
# モデルの保存 (1)
model.save('test')
# 環境のクローズ
env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\atusi\anaconda3\envs\gymenv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 36.6     |
|    ep_rew_mean     | 337      |
| time/              |          |
|    fps             | 111      |
|    iterations      | 1        |
|    time_elapsed    | 18       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 39.6        |
|    ep_rew_mean          | 365         |
| time/                   |             |
|    fps                  | 111         |
|    iterations           | 2           |
|    time_elapsed         | 36          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004629122 |
|    clip_fraction        | 0.0778      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42       |
|    explained_variance   | 0.00385     |
|    learning_rate        | 0.

ValueError: You have passed a tuple to the predict() function instead of a Numpy array or a Dict. You are probably mixing Gym API with SB3 VecEnv API: `obs, info = env.reset()` (Gym) vs `obs = vec_env.reset()` (SB3 VecEnv). See related issue https://github.com/DLR-RM/stable-baselines3/issues/1694 and documentation for more information: https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api